# 0. Revisión de los datos
#### En primer lugar, revisamos el set de datos para entender cómo están estructurados y si existen anomalías. 

## 0.1. Imports

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats.mstats as sp
%matplotlib inline

## 0.2. Cargamos data

In [31]:
train = pd.read_csv('train.csv')#, encoding='latin_1')
test = pd.read_csv('test.csv')

## 0.3. Revisamos los datos.
### 0.3.1. Set de datos train
Comenzamos chequeando el dataset train: tamaño, tipo de datos y cantidad de nulos.

In [32]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Vemos que se trata de una muestra de un dataset más grande pues los valores de id no coinciden con los valores del índice. Además, ya se observa que al menos keyword y location poseen nulos.

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [34]:
7552*100/7613, 5080*100/7613

(99.19873899908052, 66.72796532247472)

Keyword presenta cerca de un 1% de nulos, location cerca de 33%, el resto ninguno. Keyword, location y text son objetos con texto, las otras dos variables constan de enteros.

In [35]:
train.describe(include='all')

,id,keyword,location,text,target
count,7613.000000,7552,5080,7613,7613.00000
unique,NaN,221,3341,7503,NaN
top,NaN,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...,NaN
freq,NaN,45,104,10,NaN
mean,5441.934848,NaN,NaN,NaN,0.42966
std,3137.116090,NaN,NaN,NaN,0.49506
min,1.000000,NaN,NaN,NaN,0.00000
25%,2734.000000,NaN,NaN,NaN,0.00000
50%,5408.000000,NaN,NaN,NaN,0.00000
75%,8146.000000,NaN,NaN,NaN,1.00000


Una estadística descriptiva inicial revela que en keyword se encuentran sólo 221 valores únicos, con los cual podría transformarse en una variable categórica si el especio en memoria fuera un problema, pero no es el caso. El valor que aparece con mayor frecuencia es "fatalities", pero con solo 45 casos, lo cual sugiere que las keywords se reparten bastante uniformemente respecto de los tweets. Nos preguntamos, ¿cómo fue generada esta variable? ¿Con qué criterio se asignó cada keyword a cada tweet? ¿Los términos fueron tomados del texto de los tweets? En Kaggle no figuran las respuestas a estas preguntas. Sölo se menciona el origen de los datos, pero ya no se los encuentra en la página de internet que citan como fuente (www.XXXXXXXX). Una búsqueda en google (XXXXXXXX primeros resultados) no nos disipa la duda.
Por su parte, la variable location cuenta con 3341 valores únicos, y su valor más frecuente es USA, con 104 ocurrencias. En principio parece que las ubicaciones son muy variables.
Respecto de text notamos, al comparar valores únicos con el tamaño de la muestra, que cerca de 100 tweets se encuentran repetidos, siendo el más repetido (10 veces) el que se observa en la Tabla X. Más adelante analizaremos algunos de estos tweets en particular.
La clasificación de los teewts según la variable target ha sido realizada para el total de ellos. La media indica que cerca del 43% de los tweets corresponde con una emergencia real, es decir que ambas categorías están representadas de manera pareja bastante pareja.

### 0.3.1. Set de datos test
Realizamos similar análisis chequeando el dataset de testeo (test).

In [36]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


Los porcentajes de nulos en keyword y location son similares a los de train:

In [116]:
3237*100/3263, 2158*100/3263

(99.20318725099601, 66.13545816733068)

In [38]:
test.describe(include='all')

,id,keyword,location,text
count,3263.000000,3237,2158,3263
unique,NaN,221,1602,3243
top,NaN,deluged,New York,11-Year-Old Boy Charged With Manslaughter of T...
freq,NaN,23,38,3
mean,5427.152927,NaN,NaN,NaN
std,3146.427221,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN
25%,2683.000000,NaN,NaN,NaN
50%,5500.000000,NaN,NaN,NaN
75%,8176.000000,NaN,NaN,NaN


En este caso, vemos que la gran mayoría de los tweets tienen asignada una keyword, no asi una location. Las keywords son, nuevamente, 221 en total, lo que sugiere chequear que sean las mismas. 

In [39]:
train['keyword'].drop_duplicates().isin(test['keyword'].drop_duplicates()).value_counts()

True    222
Name: keyword, dtype: int64

Efectivamente, son las mismas.
Respecto del resto de las variables, el análisis es similar al de train, salvo para el valor más frecuente de location. Llama la atención que en train sea un país (USA), y en test una ciudad (New York). Esto merece un análisis más pormenorizado, con el objeto de ver si es necesario, en la medida de lo posible, estandarizar location, de modo de tener sólo ciudades, países, o lo que más convenga.

Pasaremos a analizar la distribución de las variables independientemente del target.